In [1]:
import tensorflow as tf

hello=tf.constant("Hello,world!")
sess=tf.Session()
res=sess.run(hello)
sess.close()
print res

Hello,world!


In [2]:
# -*- coding:utf8 -*-

2. 我们使用Tensorflow，计算((a+b)*c)^2/a，然后求平方根。看代码：

In [3]:
a=tf.placeholder(tf.float16)
b=tf.placeholder(tf.float16)
c=tf.placeholder(tf.float16)

d=tf.add(a,b)
e=tf.multiply(d,c)
f=tf.pow(e,2)
g=tf.divide(f,a) 
h=tf.sqrt(g) #平方跟

sess=tf.Session()
feed_dict={a:1,b:2,c:3}
res=sess.run(h,feed_dict=feed_dict)
sess.close()
print res

9.0


这里让a=1，b=2，c=3，如果输出9.0，证明运行成功。

Tensorflow做计算的方法是，先把计算的式子构建一个图，然后把这个图和赋值在cpu上一起运行，计算速度比较快。

In [4]:
hello = tf.constant('Hello,world!', dtype=tf.string)
#'Hello,world!'是常量初始值；tf.string是常量类型，可以省略。常量和变量都可以去构建Tensorflow中的图。
a = tf.Variable(10, dtype=tf.int32)
#10是变量初始值，tf.int32是变量的类型。

1、创建一个2*3的矩阵，并让所有元素的值为0.（类型为tf.float）

In [6]:
a=tf.zeros([2,3],tf.float32)

2、创建一个3*4的矩阵，并让所有元素的值为1.

In [11]:
b=tf.ones([3,4])

3、创建一个1*10的矩阵，使用2来填充。（类型为tf.int32，可忽略）

In [13]:
c=tf.constant(2,tf.int32,[1,10])

4、创建一个1*10的矩阵，其中的元素符合正态分布，平均值是20，标准偏差是3.

In [14]:
d=tf.random_normal([1,10],20,3)

上面所有的值都可以用来初始化变量。例如用0.01来填充一个1*2的矩阵来初始化一个叫bias的变量。

In [15]:
bias=tf.Variable(tf.zeros([1,2])+0.01)

In [7]:
sess=tf.Session()
print sess.run(a)
#想查看这些量具体的值，可以在Session中执行它并输出。

[[ 0.  0.  0.]
 [ 0.  0.  0.]]


In [9]:
import numpy as np
x_data=np.linspace(-1,1,300)[:,np.newaxis] # -1到1等分300份形成的二维矩阵
noise=np.random.normal(0,0.05,x_data.shape)# 噪音，形状同x_data在0-0.05符合正态分布的小数
y_data=np.square(x_data)-0.5+noise
#输入层 （1个神经元）
with tf.name_scope('input_layer'): #输入层。将这两个变量放到input_layer作用域下，tensorboard会把他们放在一个图形里面
    xs = tf.placeholder(tf.float32, [None, 1], name = 'x_input') # xs起名x_input，会在图形上显示
    ys = tf.placeholder(tf.float32, [None, 1], name = 'y_input') # ys起名y_input，会在图形上显示
#隐含层 （10个神经元）
with tf.name_scope('hidden_layer'):
    with tf.name_scope('weight'): #权重
        W1 = tf.Variable(tf.random_normal([1,10]),name='W1')
  #  w1=tf.Variable(tf.random_normal([1,10]))# 权重，1*10的矩阵，并用符合正态分布的随机数填充
    with tf.name_scope('bias'): #权重
        b1=tf.Variable(tf.zeros([1,10])+0.1,name='b1')# 偏置，1*10的矩阵，使用0.1填充
    with tf.name_scope('out1'): #权重
        Wx_plus_b1=tf.matmul(xs,W1)+b1
        output1=tf.nn.relu(Wx_plus_b1)
# 输出层（1个神经元
with tf.name_scope('output_layer'):
    with tf.name_scope('weight'): #权重
        W2=tf.Variable(tf.random_normal([10,1]),name='W2')
    with tf.name_scope('bias'): #权重
        b2=tf.Variable(tf.zeros([1,1])+0.1,name='b2')
    with tf.name_scope('out2'): #权重
        Wx_plus_b2=tf.matmul(output1,W2)+b2
        output2=Wx_plus_b2
# 0:列---第一维度tf.scalar_summary('loss',loss) #命名和赋值 
# 1:行---第二维度
with tf.name_scope('loss'): 
    loss=tf.reduce_mean(tf.reduce_sum(tf.square(ys-output2),reduction_indices=[1]))
    tf.summary.scalar('loss',loss)
    
# 在第一维上，偏差平方后求和，再求平均值，来计算损失
with tf.name_scope('train'): 
    train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
# 使用梯度下降法，设置步长0.1，来最小化损失
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)

merged = tf.summary.merge_all() #将图形、训练过程等数据合并在一起
writer = tf.summary.FileWriter('train/',sess.graph) #将训练日志写入到logs文件夹下

#训练
for i in range(100):
    summary,_,loss_value=sess.run([merged,train_step,loss],feed_dict={xs:x_data,ys:y_data})
    # if(i%50==0): #每50次写一次日志
    #     result = sess.run(merged,feed_dict={xs:x_data,ys:y_data}) #计算需要写入的日志数据
    writer.add_summary(summary,i) #将日志数据写入文件
        
print(loss_value)

print sess.run(W2)
#print W2.dtype
#print sess.run(W2[3][0])

InvalidArgumentError: You must feed a value for placeholder tensor 'input_layer_4/x_input' with dtype float
	 [[Node: input_layer_4/x_input = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'input_layer_4/x_input', defined at:
  File "/home/kaido/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/kaido/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/zmq/eventloop/

In [35]:
print sess.run(w1)
print sess.run(w1[0][1])

[[-1.25438368  0.38643748  1.76592386  0.80964911 -0.3027485  -0.18774365
   0.64830774 -0.42942187  0.76283538  1.29136205]]
0.386437


In [7]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
mnist=input_data.read_data_sets("MNIST_data/",one_hot=True)
#输入
with tf.name_scope('input_layer'): 
    x=tf.placeholder(tf.float32,[None,784])#一行一幅图像
    y_=tf.placeholder(tf.float32,[None,10])#输入占位符（这张手写数字具体代表的值，0-9对应矩阵的10个位置）
    
with tf.name_scope('output_layer'):
    # 计算分类softmax会将xW+b分成10类，对应0-9
    with tf.name_scope('weight'): #权重
        W=tf.Variable(tf.zeros([784,10]))
        tf.summary.histogram('output_layer/weight', W)
    with tf.name_scope('bias'): #偏置
        b=tf.Variable(tf.zeros([10]))
        tf.summary.histogram('output_layer/bias', b)
    with tf.name_scope('Wx_plus_b'): #净输入
        Wx_plus_b=tf.matmul(x,W)+b
        tf.summary.histogram('output_layer/Wx_plus_b',Wx_plus_b)
y=tf.nn.softmax(Wx_plus_b)

#计算偏差和
with tf.name_scope('cross_entropy'): #损失
    cross_entropy=-tf.reduce_sum(y_*tf.log(y))
    loss=tf.reduce_mean(cross_entropy,name='xentropy_mean')
    tf.summary.scalar('loss',loss)
# 使用梯度下降法（步长0.01），来使偏差和最小
with tf.name_scope('train'): #训练过程
    train_step=tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
merged = tf.summary.merge_all() #将图形、训练过程等数据合并在一起
#writer = tf.summary.FileWriter('logs',sess.graph) #将训练日志写入到logs文件夹下
train_writer = tf.summary.FileWriter('train', sess.graph)

for i in range(10):
    batch_xs,batch_ys=mnist.train.next_batch(100) # 随机取100个手写数字图片
    _,loss=sess.run([train_step,loss],feed_dict={x:batch_xs,y_:batch_ys})
    res=sess.run(merged,feed_dict={x:batch_xs,y_:batch_ys})
    train_writer.add_summary(res,i) #将日志数据写入文件
    print loss
    
# 计算训练精度
#http://stackoverflow.com/questions/41708572/tensorflow-questions-regarding-tf-argmax-and-tf-equal
#argmax返回某一个轴上最大值的索引 1：行 0：列
#tf.argmax(y,1)y是 none行10列 ，所以返回的是这张图像被识别的数字值
#tf.equal(tf.argmax(pred, 1),tf.argmax(y, 1))返回评估将给出的张量array(1,1,1,1,1,1)
correct_prediction=tf.equal(tf.argmax(y,1),tf.argmax(y_,1))
# http://blog.csdn.net/lenbow/article/details/52152766
#tf.cast(x, dtype, name=None)	将x或者x.values转换为dtype
accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
print(sess.run(accuracy,feed_dict={x:mnist.test.images,y_:mnist.test.labels}))

Extracting MNIST_data/train-images-idx3-ubyte.gz


Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


InvalidArgumentError: You must feed a value for placeholder tensor 'input_layer/Placeholder' with dtype float
	 [[Node: input_layer/Placeholder = Placeholder[dtype=DT_FLOAT, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op u'input_layer/Placeholder', defined at:
  File "/home/kaido/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/kaido/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/zmq/even

In [11]:
print sess.run(tf.reduce_mean(tf.cast([1,1,1,1,0,1],tf.float32)))

0.833333


我们的模型训练出来想给别人用，或者是我今天训练不完，明天想接着训练，怎么办？这就需要模型的保存与读取。看代码：

In [27]:
from __future__ import division
import tensorflow as tf
import numpy as np
import os
x_data=np.linspace(-1,1,300)[:,np.newaxis] 
noise=np.random.normal(0,0.05,x_data.shape)
y_data=np.square(x_data)-0.5+noise
#输入层 （1个神经元）
xs = tf.placeholder(tf.float32, [None, 1]) 
ys = tf.placeholder(tf.float32, [None, 1]) 
#隐含层 （10个神经元）
W1 = tf.Variable(tf.random_normal([1,10]))
b1=tf.Variable(tf.zeros([1,10])+0.1)
Wx_plus_b1=tf.matmul(xs,W1)+b1
output1=tf.nn.relu(Wx_plus_b1)
# 输出层（1个神经元
W2=tf.Variable(tf.random_normal([10,1]))
b2=tf.Variable(tf.zeros([1,1])+0.1)
Wx_plus_b2=tf.matmul(output1,W2)+b2
output2=Wx_plus_b2

loss=tf.reduce_mean(tf.reduce_sum(tf.square(ys-output2),reduction_indices=[1]))
train_step=tf.train.GradientDescentOptimizer(0.1).minimize(loss)
#模型保存加载工具
saver=tf.train.Saver()
#判断模型保存路径是否存在，不存在就创建
if not os.path.exists('tmp/'):
    os.mkdir('tmp/')
sess=tf.Session()

if os.path.exists('tmp/checkpoint'):#判断模型是否存在
    saver.restore(sess,'tmp/model.ckpt') #存在就从模型中恢复变量
else:
    init=tf.global_variables_initializer()#不存在就初始化变量
    sess.run(init)

for i in range(1000):
    _,loss_value=sess.run([train_step,loss],feed_dict={xs:x_data,ys:y_data})
    if(i%50==0):
        save_path=saver.save(sess,'tmp/model.ckpt')#保存模型到tmp/model.ckpt，注意一定要有一层文件夹，否则保存不成功！！！
        print("模型保存：%s 当前训练损失：%s"%(save_path,loss_value))

模型保存：tmp/model.ckpt 当前训练损失：1.25403


模型保存：tmp/model.ckpt 当前训练损失：0.0105221


模型保存：tmp/model.ckpt 当前训练损失：0.00596945


模型保存：tmp/model.ckpt 当前训练损失：0.00432875


模型保存：tmp/model.ckpt 当前训练损失：0.00364213


...

模型保存：tmp/model.ckpt 当前训练损失：0.00289579


模型保存：tmp/model.ckpt 当前训练损失：0.00288812


模型保存：tmp/model.ckpt 当前训练损失：0.00288138


模型保存：tmp/model.ckpt 当前训练损失：0.00287621


模型保存：tmp/model.ckpt 当前训练损失：0.00287168


首先，下载一张png格式的图片（注意：只支持png格式），命名为1.png。然后，打开PythonShell，输入以下代码：

In [39]:
import tensorflow as tf

# 获取图片数据
file = open('1.png', 'rb')
data = file.read()
file.close()

# 图片处理
image = tf.image.decode_png(data, channels=4)
image = tf.expand_dims(image, 0)

# 添加到日志中
sess = tf.Session()
writer = tf.summary.FileWriter('logs')
summary_op = tf.summary.image("image1", image)

# 运行并写入日志
summary = sess.run(summary_op)
writer.add_summary(summary)

# 关闭
writer.close()
sess.close()

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 1132, in get_records


    return _fixed_getinnerframes(etb, number_of_lines_of_context, tb_offset)
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 313, in wrapped
    return f(*args, **kwargs)
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 358, in _fixed_getinnerframes
    records = fix_frame_records_filenames(inspect.getinnerframes(etb, context))
  File "/home/kaido/anaconda2/lib/python2.7/inspect.py", line 1048, in getinnerframes
    framelist.append((tb.tb_frame,) + getframeinfo(tb, context))


  File "/home/kaido/anaconda2/lib/python2.7/inspect.py", line 1012, in getframeinfo
    lines, lnum = findsource(frame)
  File "/home/kaido/anaconda2/lib/python2.7/site-packages/IPython/core/ultratb.py", line 182, in findsource
    lines = linecache.getlines(file, globals_dict)
  File "/home/kaido/anaconda2/lib/python2.7/linecache.py", line 41, in getlines
    return updatecache(filename, module_globals)
  File "/home/kaido/anaconda2/lib/python2.7/linecache.py", line 132, in updatecache
    lines = fp.readlines()
KeyboardInterrupt


IndexError: string index out of range

In [40]:
#参考资料：《Tensorflow: How to Display Custom Images in Tensorboard (e.g. Matplotlib Plots)》：http://stackoverflow.com/questions/38543850/tensorflow-how-to-display-custom-images-in-tensorboard-e-g-matplotlib-plots